In [30]:
# 直接装饰，发现新函数只在第一次被调用， 且原函数多调用一次
def dec(add):
    print('begin')
    add()
    print('end')
    return add

@dec
def add():
    print('123')

if __name__ == '__main__':
    add()
    print('-------')
    add()

begin
123
end
123
-------
123


In [32]:
# 使用内嵌包装函数确保每次新函数都被调用
# 内嵌包装函数的形参和返回值与原函数相同， 装饰函数返回内嵌包装函数
def dec(add):
    def wrapper(a, b):
        if a<0 or a>100 or b<0 or b>100:
            print('error')
        else:
            print('begin')
            print(add(a, b))
            print('end')
    return wrapper

@dec
def add(a, b):
    return a+b

if __name__ == '__main__':
    add(3, 4)
    print('-------')
    add(-1, 2)

begin
7
end
-------
error


In [37]:
# 对参数不确定的函数进行装饰
def dec(func):
    def wrapper(*args, **kwargs):
        print('begin')
        print(func(*args, **kwargs))
        print('end')
    return wrapper

@dec
def func1(a, b):
    return a+b

@dec 
def func2(a, b, c):
    return a+b+c

if __name__ == '__main__':
    func1(1,2)
    print('-------')
    func2(1,2,3)

begin
3
end
-------
begin
6
end


In [51]:
# 让装饰器带参数
# 在外层多了一层包装
def dec(arg):
    def _dec(func):
        def wrapper():
            print('before %s called [%s]' % (func.__name__, arg))
            print(func())
            print('after {0} called [{1}]'.format(func.__name__, arg))
        return wrapper
    return _dec

@dec('module1')
def func1():
    return '123'

@dec('module2')
def func2():
    return '233'
    
if __name__ == '__main__':
    func1()
    print('-----------------'*2)
    func2()

before func1 called [module1]
123
after func1 called [module1]
----------------------------------
before func2 called [module2]
233
after func2 called [module2]


In [58]:
# 让装饰器带类参数
class locker:
    def __init__(self):
        print('locker.__init__() should not be called')
        
    @staticmethod
    def acquire():
        print('locker.acquire() called')
        
    @staticmethod
    def release():
        print('locker.release() called')
        
def dec(cls):
    # cls必须实现acquire和release静态方法(不实现好像也行)
    def _dec(func):
        def wrapper():
            print('before {0} called [{1}]'.format(func.__name__, cls))
            cls.acquire()
            try:
                return func()
            finally:
                cls.release()
#             func()
        return wrapper
    return _dec

@dec(locker)
def myfunc():
    print('myfunc() called')
    
if __name__ == '__main__':
    myfunc()
    print('------------------------'*2)
    myfunc()

before myfunc called [<class '__main__.locker'>]
locker.acquire() called
myfunc() called
locker.release() called
------------------------------------------------
before myfunc called [<class '__main__.locker'>]
locker.acquire() called
myfunc() called
locker.release() called
